## Calculating similarity score using Jaro Winkler distance.

In [1]:
!pip install strsimpy

In [2]:
# importing all necessary libraries
import pandas as pd
import numpy as np
import json, collections
from pandas.io.json import json_normalize
import os

In [3]:
#importing libraries
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import statistics
import math
import os
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import re
pd.set_option('max_colwidth', 800)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
df = pd.read_csv('/content/Cases 50-91_Self medication ChatGPT.csv', encoding='ISO-8859-1')
df.head()

,Unnamed: 0,ChatGPT Response,Pharmacist response,Standard Answer,Unnamed: 4
0,50,"There are several alternative pain relief options that you can consider that may not cause drowsiness:\n\n1. Acetaminophen (Tylenol): Acetaminophen is a pain reliever that is available over-the-counter. It is generally well-tolerated and does not cause drowsiness in most people.\n\n2. Nonsteroidal anti-inflammatory drugs (NSAIDs): NSAIDs such as ibuprofen (Advil, Motrin) and naproxen (Aleve) are effective pain relievers that can reduce inflammation. They are also available over-the-counter and are generally well-tolerated, although they can cause stomach upset in some people.\n\n3. Topical pain relievers: Topical pain relievers such as creams, gels, or patches can be applied directly to the area of pain. They work by numbing the area or reducing inflammation. Some examples include lido...",Stop the medication\nRest\nTake paracetamol or ibuprofen instead.,"Make sure not to take it before driving , If it worsens refer to PCP",NaN
1,51,"There are several medications that can help relieve muscle spasms in the neck. These include:\n\n1. Muscle relaxants: Muscle relaxants are medications that can help relax muscles and reduce muscle spasms. Examples include cyclobenzaprine (Flexeril), baclofen (Lioresal), and tizanidine (Zanaflex). Muscle relaxants require a prescription from a healthcare provider.\n\n2. Nonsteroidal anti-inflammatory drugs (NSAIDs): NSAIDs such as ibuprofen (Advil, Motrin) and naproxen (Aleve) can help relieve pain and reduce inflammation associated with muscle spasms. These medications are available over-the-counter.\n\n3. Acetaminophen (Tylenol): Acetaminophen is a pain reliever that can help relieve pain associated with muscle spasms. It is available over-the-counter.\n\n4. Topical pain relievers: To...",Myonal tab and Cold water packs,"Lifestyle change, massage, heat, if it worsens or continues to happen after 2 weeks refer to PCP",NaN
2,52,"As mentioned earlier, medications that can help relieve muscle spasms in the neck include:\n\n1. Muscle relaxants: Muscle relaxants are medications that can help relax muscles and reduce muscle spasms. Examples include cyclobenzaprine (Flexeril), baclofen (Lioresal), and tizanidine (Zanaflex). Muscle relaxants require a prescription from a healthcare provider.\n\n2. Nonsteroidal anti-inflammatory drugs (NSAIDs): NSAIDs such as ibuprofen (Advil, Motrin) and naproxen (Aleve) can help relieve pain and reduce inflammation associated with muscle spasms. These medications are available over-the-counter.\n\n3. Acetaminophen (Tylenol): Acetaminophen is a pain reliever that can help relieve pain associated with muscle spasms. It is available over-the-counter.\n\n4. Topical pain relievers: Topic...",Magnesium,use NSAIDs if it worsens or continues to happen after 2 weeks refer to PCP,NaN
3,53,"There are over-the-counter ear wax removal products that are safe for children, but it's important to use them carefully and under the guidance of a healthcare provider. Some examples of over-the-counter ear wax removal products that are safe for children include:\n\n1. Debrox Earwax Removal Kit: This kit includes drops that help soften and loosen ear wax, as well as a bulb syringe to help rinse the ear canal.\n\n2. Murine Ear Drops: These drops are designed to help soften and loosen ear wax, making it easier to remove.\n\n3. Similasan Ear Wax Removal Kit: This kit includes drops that help break down and remove ear wax, as well as a bulb syringe to help rinse the ear canal.\n\nBefore using any over-the-counter ear wax removal product in a child, it's important to read and follow the in...",Its important to check with the doctor maybe there is an infection.\nibuprofen\nSaline drops,"Patients symptoms are consistent with common presentation of labyrinthitis, and the pharmacist should refer him to his PCP for further evaluation and treatment. The symptoms of hearing loss and poor balance, especially in a child with 

In [5]:
df.shape

(42, 5)

In [6]:
df.columns

Index(['Unnamed: 0', 'ChatGPT Response', 'Pharmacist response',
       'Standard Answer', 'Unnamed: 4'],
      dtype='object')

In [7]:
# Assuming your DataFrame is df
selected_columns = ['ChatGPT Response', 'Standard Answer']  # Replace 'Column1' and 'Column2' with your actual column names

# Select only the specified columns
df2 = df[selected_columns]

# Display the resulting DataFrame
df2

,ChatGPT Response,Standard Answer
0,"There are several alternative pain relief options that you can consider that may not cause drowsiness:\n\n1. Acetaminophen (Tylenol): Acetaminophen is a pain reliever that is available over-the-counter. It is generally well-tolerated and does not cause drowsiness in most people.\n\n2. Nonsteroidal anti-inflammatory drugs (NSAIDs): NSAIDs such as ibuprofen (Advil, Motrin) and naproxen (Aleve) are effective pain relievers that can reduce inflammation. They are also available over-the-counter and are generally well-tolerated, although they can cause stomach upset in some people.\n\n3. Topical pain relievers: Topical pain relievers such as creams, gels, or patches can be applied directly to the area of pain. They work by numbing the area or reducing inflammation. Some examples include lido...","Make sure not to take it before driving , If it worsens refer to PCP"
1,"There are several medications that can help relieve muscle spasms in the neck. These include:\n\n1. Muscle relaxants: Muscle relaxants are medications that can help relax muscles and reduce muscle spasms. Examples include cyclobenzaprine (Flexeril), baclofen (Lioresal), and tizanidine (Zanaflex). Muscle relaxants require a prescription from a healthcare provider.\n\n2. Nonsteroidal anti-inflammatory drugs (NSAIDs): NSAIDs such as ibuprofen (Advil, Motrin) and naproxen (Aleve) can help relieve pain and reduce inflammation associated with muscle spasms. These medications are available over-the-counter.\n\n3. Acetaminophen (Tylenol): Acetaminophen is a pain reliever that can help relieve pain associated with muscle spasms. It is available over-the-counter.\n\n4. Topical pain relievers: To...","Lifestyle change, massage, heat, if it worsens or continues to happen after 2 weeks refer to PCP"
2,"As mentioned earlier, medications that can help relieve muscle spasms in the neck include:\n\n1. Muscle relaxants: Muscle relaxants are medications that can help relax muscles and reduce muscle spasms. Examples include cyclobenzaprine (Flexeril), baclofen (Lioresal), and tizanidine (Zanaflex). Muscle relaxants require a prescription from a healthcare provider.\n\n2. Nonsteroidal anti-inflammatory drugs (NSAIDs): NSAIDs such as ibuprofen (Advil, Motrin) and naproxen (Aleve) can help relieve pain and reduce inflammation associated with muscle spasms. These medications are available over-the-counter.\n\n3. Acetaminophen (Tylenol): Acetaminophen is a pain reliever that can help relieve pain associated with muscle spasms. It is available over-the-counter.\n\n4. Topical pain relievers: Topic...",use NSAIDs if it worsens or continues to happen after 2 weeks refer to PCP
3,"There are over-the-counter ear wax removal products that are safe for children, but it's important to use them carefully and under the guidance of a healthcare provider. Some examples of over-the-counter ear wax removal products that are safe for children include:\n\n1. Debrox Earwax Removal Kit: This kit includes drops that help soften and loosen ear wax, as well as a bulb syringe to help rinse the ear canal.\n\n2. Murine Ear Drops: These drops are designed to help soften and loosen ear wax, making it easier to remove.\n\n3. Similasan Ear Wax Removal Kit: This kit includes drops that help break down and remove ear wax, as well as a bulb syringe to help rinse the ear canal.\n\nBefore using any over-the-counter ear wax removal product in a child, it's important to read and follow the in...","Patients symptoms are consistent with common presentation of labyrinthitis, and the pharmacist should refer him to his PCP for further evaluation and treatment. The symptoms of hearing loss and poor balance, especially in a child with a recent common cold, suggest viral etiology. Viral infections are a common trigger for labyrinthitis. PJs PCP may conduct testing and refer the child to a specialist. In some cases, corticosteroids or antiviral medications may be prescribed."
4,"If your 3-year-old, 

In [8]:
# Cleaning Raw tweets
def clean_text(text):

    #remove emails
    text = ' '.join([i for i in text.split() if '@' not in i])

    #remove web address
    text = re.sub('http[s]?://\S+', '', text)

    #remove brackets and parantheses (and removing the text inside the brackets and parantheses)
    text = re.sub("[\(\[].*?[\)\]]", "", text)

    #remove punctuations
    text = re.sub(r'[^\w\s]', ' ', text)

    #Filter to allow only alphabets and numbers
    text = re.sub(r'[^a-zA-Z0-9\']', ' ', text)

    #Remove Unicode characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    #remove double spaces
    text = re.sub('\s+', ' ', text)

    return text

In [9]:
# applying that into function.
df2["ChatGPT_Response"] = df2['ChatGPT Response'].apply(lambda x: clean_text(x))
df2["Standard_Answer"] = df2['Standard Answer'].apply(lambda x: clean_text(x))
df2

<ipython-input-9-ed8f55050a9d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["ChatGPT_Response"] = df2['ChatGPT Response'].apply(lambda x: clean_text(x))
<ipython-input-9-ed8f55050a9d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Standard_Answer"] = df2['Standard Answer'].apply(lambda x: clean_text(x))


,ChatGPT Response,Standard Answer,ChatGPT_Response,Standard_Answer
0,"There are several alternative pain relief options that you can consider that may not cause drowsiness:\n\n1. Acetaminophen (Tylenol): Acetaminophen is a pain reliever that is available over-the-counter. It is generally well-tolerated and does not cause drowsiness in most people.\n\n2. Nonsteroidal anti-inflammatory drugs (NSAIDs): NSAIDs such as ibuprofen (Advil, Motrin) and naproxen (Aleve) are effective pain relievers that can reduce inflammation. They are also available over-the-counter and are generally well-tolerated, although they can cause stomach upset in some people.\n\n3. Topical pain relievers: Topical pain relievers such as creams, gels, or patches can be applied directly to the area of pain. They work by numbing the area or reducing inflammation. Some examples include lido...","Make sure not to take it before driving , If it worsens refer to PCP",There are several alternative pain relief options that you can consider that may not cause drowsiness 1 Acetaminophen Acetaminophen is a pain reliever that is available over the counter It is generally well tolerated and does not cause drowsiness in most people 2 Nonsteroidal anti inflammatory drugs NSAIDs such as ibuprofen and naproxen are effective pain relievers that can reduce inflammation They are also available over the counter and are generally well tolerated although they can cause stomach upset in some people 3 Topical pain relievers Topical pain relievers such as creams gels or patches can be applied directly to the area of pain They work by numbing the area or reducing inflammation Some examples include lidocaine patches capsaicin cream and diclofenac gel 4 Physical therapy ...,Make sure not to take it before driving If it worsens refer to PCP
1,"There are several medications that can help relieve muscle spasms in the neck. These include:\n\n1. Muscle relaxants: Muscle relaxants are medications that can help relax muscles and reduce muscle spasms. Examples include cyclobenzaprine (Flexeril), baclofen (Lioresal), and tizanidine (Zanaflex). Muscle relaxants require a prescription from a healthcare provider.\n\n2. Nonsteroidal anti-inflammatory drugs (NSAIDs): NSAIDs such as ibuprofen (Advil, Motrin) and naproxen (Aleve) can help relieve pain and reduce inflammation associated with muscle spasms. These medications are available over-the-counter.\n\n3. Acetaminophen (Tylenol): Acetaminophen is a pain reliever that can help relieve pain associated with muscle spasms. It is available over-the-counter.\n\n4. Topical pain relievers: To...","Lifestyle change, massage, heat, if it worsens or continues to happen after 2 weeks refer to PCP",There are several medications that can help relieve muscle spasms in the neck These include 1 Muscle relaxants Muscle relaxants are medications that can help relax muscles and reduce muscle spasms Examples include cyclobenzaprine baclofen and tizanidine Muscle relaxants require a prescription from a healthcare provider 2 Nonsteroidal anti inflammatory drugs NSAIDs such as ibuprofen and naproxen can help relieve pain and reduce inflammation associated with muscle spasms These medications are available over the counter 3 Acetaminophen Acetaminophen is a pain reliever that can help relieve pain associated with muscle spasms It is available over the counter 4 Topical pain relievers Topical pain relievers such as creams gels or patches can be applied directly to the area of pain They work b...,Lifestyle change massage heat if it worsens or continues to happen after 2 weeks refer to PCP
2,"As mentioned earlier, medications that can help relieve muscle spasms in the neck include:\n\n1. Muscle relaxants: Muscle relaxants are medications that can help relax muscles and reduce muscle spasms. Examples include cyclobenzaprine (Flexeril), baclofen (Lioresal), and tizanidine (Zanaflex). Muscle relaxants require a prescription from a healthcare provider.\n\n2. Nonsteroidal anti-infl

In [10]:
df2.drop(["ChatGPT Response", "Standard Answer"], axis=1, inplace=True)
df2.head()

<ipython-input-10-ad20dad2869b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop(["ChatGPT Response", "Standard Answer"], axis=1, inplace=True)


,ChatGPT_Response,Standard_Answer
0,There are several alternative pain relief options that you can consider that may not cause drowsiness 1 Acetaminophen Acetaminophen is a pain reliever that is available over the counter It is generally well tolerated and does not cause drowsiness in most people 2 Nonsteroidal anti inflammatory drugs NSAIDs such as ibuprofen and naproxen are effective pain relievers that can reduce inflammation They are also available over the counter and are generally well tolerated although they can cause stomach upset in some people 3 Topical pain relievers Topical pain relievers such as creams gels or patches can be applied directly to the area of pain They work by numbing the area or reducing inflammation Some examples include lidocaine patches capsaicin cream and diclofenac gel 4 Physical therapy ...,Make sure not to take it before driving If it worsens refer to PCP
1,There are several medications that can help relieve muscle spasms in the neck These include 1 Muscle relaxants Muscle relaxants are medications that can help relax muscles and reduce muscle spasms Examples include cyclobenzaprine baclofen and tizanidine Muscle relaxants require a prescription from a healthcare provider 2 Nonsteroidal anti inflammatory drugs NSAIDs such as ibuprofen and naproxen can help relieve pain and reduce inflammation associated with muscle spasms These medications are available over the counter 3 Acetaminophen Acetaminophen is a pain reliever that can help relieve pain associated with muscle spasms It is available over the counter 4 Topical pain relievers Topical pain relievers such as creams gels or patches can be applied directly to the area of pain They work b...,Lifestyle change massage heat if it worsens or continues to happen after 2 weeks refer to PCP
2,As mentioned earlier medications that can help relieve muscle spasms in the neck include 1 Muscle relaxants Muscle relaxants are medications that can help relax muscles and reduce muscle spasms Examples include cyclobenzaprine baclofen and tizanidine Muscle relaxants require a prescription from a healthcare provider 2 Nonsteroidal anti inflammatory drugs NSAIDs such as ibuprofen and naproxen can help relieve pain and reduce inflammation associated with muscle spasms These medications are available over the counter 3 Acetaminophen Acetaminophen is a pain reliever that can help relieve pain associated with muscle spasms It is available over the counter 4 Topical pain relievers Topical pain relievers such as creams gels or patches can be applied directly to the area of pain They work by n...,use NSAIDs if it worsens or continues to happen after 2 weeks refer to PCP
3,There are over the counter ear wax removal products that are safe for children but it s important to use them carefully and under the guidance of a healthcare provider Some examples of over the counter ear wax removal products that are safe for children include 1 Debrox Earwax Removal Kit This kit includes drops that help soften and loosen ear wax as well as a bulb syringe to help rinse the ear canal 2 Murine Ear Drops These drops are designed to help soften and loosen ear wax making it easier to remove 3 Similasan Ear Wax Removal Kit This kit includes drops that help break down and remove ear wax as well as a bulb syringe to help rinse the ear canal Before using any over the counter ear wax removal product in a child it s important to read and follow the instructions carefully If you ...,Patients symptoms are consistent with common presentation of labyrinthitis and the pharmacist should refer him to his PCP for further evaluation and treatment The symptoms of hearing loss and poor balance especially in a child with a recent common cold suggest viral etiology Viral infections are a common trigger for labyrinthitis PJ s PCP may conduct testing and refer the child to a specialist In some cases corticosteroids or antiviral medications may be prescribed
4,If your 3 year old 7 kg boy is experiencing mild t

In [11]:
from strsimpy.cosine import Cosine
cosine = Cosine(2)
df2["p0"] = df2["ChatGPT_Response"].apply(lambda s: cosine.get_profile(s))
df2["p1"] = df2["Standard_Answer"].apply(lambda s: cosine.get_profile(s))
df2["cosine_sim"] = [cosine.similarity_profiles(p0,p1) for p0,p1 in zip(df2["p0"],df2["p1"])]

<ipython-input-11-8dfc7c31c1cb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["p0"] = df2["ChatGPT_Response"].apply(lambda s: cosine.get_profile(s))
<ipython-input-11-8dfc7c31c1cb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["p1"] = df2["Standard_Answer"].apply(lambda s: cosine.get_profile(s))


In [12]:
df2.drop(["p0", "p1"], axis=1, inplace=True)
df2.head()

,ChatGPT_Response,Standard_Answer,cosine_sim
0,There are several alternative pain relief options that you can consider that may not cause drowsiness 1 Acetaminophen Acetaminophen is a pain reliever that is available over the counter It is generally well tolerated and does not cause drowsiness in most people 2 Nonsteroidal anti inflammatory drugs NSAIDs such as ibuprofen and naproxen are effective pain relievers that can reduce inflammation They are also available over the counter and are generally well tolerated although they can cause stomach upset in some people 3 Topical pain relievers Topical pain relievers such as creams gels or patches can be applied directly to the area of pain They work by numbing the area or reducing inflammation Some examples include lidocaine patches capsaicin cream and diclofenac gel 4 Physical therapy ...,Make sure not to take it before driving If it worsens refer to PCP,0.479287
1,There are several medications that can help relieve muscle spasms in the neck These include 1 Muscle relaxants Muscle relaxants are medications that can help relax muscles and reduce muscle spasms Examples include cyclobenzaprine baclofen and tizanidine Muscle relaxants require a prescription from a healthcare provider 2 Nonsteroidal anti inflammatory drugs NSAIDs such as ibuprofen and naproxen can help relieve pain and reduce inflammation associated with muscle spasms These medications are available over the counter 3 Acetaminophen Acetaminophen is a pain reliever that can help relieve pain associated with muscle spasms It is available over the counter 4 Topical pain relievers Topical pain relievers such as creams gels or patches can be applied directly to the area of pain They work b...,Lifestyle change massage heat if it worsens or continues to happen after 2 weeks refer to PCP,0.584904
2,As mentioned earlier medications that can help relieve muscle spasms in the neck include 1 Muscle relaxants Muscle relaxants are medications that can help relax muscles and reduce muscle spasms Examples include cyclobenzaprine baclofen and tizanidine Muscle relaxants require a prescription from a healthcare provider 2 Nonsteroidal anti inflammatory drugs NSAIDs such as ibuprofen and naproxen can help relieve pain and reduce inflammation associated with muscle spasms These medications are available over the counter 3 Acetaminophen Acetaminophen is a pain reliever that can help relieve pain associated with muscle spasms It is available over the counter 4 Topical pain relievers Topical pain relievers such as creams gels or patches can be applied directly to the area of pain They work by n...,use NSAIDs if it worsens or continues to happen after 2 weeks refer to PCP,0.511657
3,There are over the counter ear wax removal products that are safe for children but it s important to use them carefully and under the guidance of a healthcare provider Some examples of over the counter ear wax removal products that are safe for children include 1 Debrox Earwax Removal Kit This kit includes drops that help soften and loosen ear wax as well as a bulb syringe to help rinse the ear canal 2 Murine Ear Drops These drops are designed to help soften and loosen ear wax making it easier to remove 3 Similasan Ear Wax Removal Kit This kit includes drops that help break down and remove ear wax as well as a bulb syringe to help rinse the ear canal Before using any over the counter ear wax removal product in a child it s important to read and follow the instructions carefully If you ...,Patients symptoms are consistent with common presentation of labyrinthitis and the pharmacist should refer him to his PCP for further evaluation and treatment The symptoms of hearing loss and poor balance especially in a child with a recent common cold suggest viral etiology Viral infections are a common trigger for labyrinthitis PJ s PCP may conduct testing and refer the child to a specialist In some cases corticosteroids or antiviral medications may be prescribed,0.762893
4,If 

In [13]:
df2

,ChatGPT_Response,Standard_Answer,cosine_sim
0,There are several alternative pain relief options that you can consider that may not cause drowsiness 1 Acetaminophen Acetaminophen is a pain reliever that is available over the counter It is generally well tolerated and does not cause drowsiness in most people 2 Nonsteroidal anti inflammatory drugs NSAIDs such as ibuprofen and naproxen are effective pain relievers that can reduce inflammation They are also available over the counter and are generally well tolerated although they can cause stomach upset in some people 3 Topical pain relievers Topical pain relievers such as creams gels or patches can be applied directly to the area of pain They work by numbing the area or reducing inflammation Some examples include lidocaine patches capsaicin cream and diclofenac gel 4 Physical therapy ...,Make sure not to take it before driving If it worsens refer to PCP,0.479287
1,There are several medications that can help relieve muscle spasms in the neck These include 1 Muscle relaxants Muscle relaxants are medications that can help relax muscles and reduce muscle spasms Examples include cyclobenzaprine baclofen and tizanidine Muscle relaxants require a prescription from a healthcare provider 2 Nonsteroidal anti inflammatory drugs NSAIDs such as ibuprofen and naproxen can help relieve pain and reduce inflammation associated with muscle spasms These medications are available over the counter 3 Acetaminophen Acetaminophen is a pain reliever that can help relieve pain associated with muscle spasms It is available over the counter 4 Topical pain relievers Topical pain relievers such as creams gels or patches can be applied directly to the area of pain They work b...,Lifestyle change massage heat if it worsens or continues to happen after 2 weeks refer to PCP,0.584904
2,As mentioned earlier medications that can help relieve muscle spasms in the neck include 1 Muscle relaxants Muscle relaxants are medications that can help relax muscles and reduce muscle spasms Examples include cyclobenzaprine baclofen and tizanidine Muscle relaxants require a prescription from a healthcare provider 2 Nonsteroidal anti inflammatory drugs NSAIDs such as ibuprofen and naproxen can help relieve pain and reduce inflammation associated with muscle spasms These medications are available over the counter 3 Acetaminophen Acetaminophen is a pain reliever that can help relieve pain associated with muscle spasms It is available over the counter 4 Topical pain relievers Topical pain relievers such as creams gels or patches can be applied directly to the area of pain They work by n...,use NSAIDs if it worsens or continues to happen after 2 weeks refer to PCP,0.511657
3,There are over the counter ear wax removal products that are safe for children but it s important to use them carefully and under the guidance of a healthcare provider Some examples of over the counter ear wax removal products that are safe for children include 1 Debrox Earwax Removal Kit This kit includes drops that help soften and loosen ear wax as well as a bulb syringe to help rinse the ear canal 2 Murine Ear Drops These drops are designed to help soften and loosen ear wax making it easier to remove 3 Similasan Ear Wax Removal Kit This kit includes drops that help break down and remove ear wax as well as a bulb syringe to help rinse the ear canal Before using any over the counter ear wax removal product in a child it s important to read and follow the instructions carefully If you ...,Patients symptoms are consistent with common presentation of labyrinthitis and the pharmacist should refer him to his PCP for further evaluation and treatment The symptoms of hearing loss and poor balance especially in a child with a recent common cold suggest viral etiology Viral infections are a common trigger for labyrinthitis PJ s PCP may conduct testing and refer the child to a specialist In some cases corticosteroids or antiviral medications may be prescribed,0.762893
4,If 

In [14]:
df2.to_csv('CHATGPT VS standard similarity.csv')